安装mxnet和gluonnlp

In [4]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python
!pip install --upgrade gluonnlp

Looking in links: https://dist.mxnet.io/python
     |████████████████████████████████| 695.0MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 256kB 8.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470041 sha256=8cad9e2814410951391b2c7a987cd4a0176a9a4ec57a779a8219e03dd45011e3
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
import os
os.chdir("/content/gdrive/My Drive/finetune")
!ls

bert	    finetune.ipynb   labeledTrainData.tsv  testDataset.tsv  try0.tsv
bert.ipynb  inference.ipynb  output		   test.params	    try.tsv


导入必要的库

In [7]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
# this notebook assumes that all required scripts are already
# downloaded from the corresponding tutorial webpage on http://gluon-nlp.mxnet.io

from bert import data

nlp.utils.check_version('0.8.1')

设置环境

In [8]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

导入BERT_BASE模型

In [9]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                        dataset_name='book_corpus_wiki_en_uncased',
                        pretrained=True, ctx=ctx, use_pooler=True,
                        use_decoder=False, use_classifier=False)
print(bert_base)

Vocab file is not found. Downloading.
BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttentionCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
      )
      (1): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (at

将模型用于分类

In [10]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

导入微调后的模型

In [16]:
filename="./base_2class.params"
nlp.utils.load_parameters(bert_classifier,filename)

推断

In [17]:
tsv_try = io.open('try0.tsv', encoding='utf-8')
print(tsv_try.readline())
print(tsv_try.readline())
sample_splitter = nlp.data.Splitter('\n')
field_separator = nlp.data.Splitter('\t')
data_try_row = nlp.data.TSVDataset(filename='try0.tsv',
                sample_splitter=sample_splitter,
                field_separator=field_separator,
                )
batch_size=32
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = similar)]
all_labels = ["0", "1"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                        class_labels=all_labels,
                        has_label=True,
                        pad=True,
                        pair=pair)
data_try=data_try_row.transform(transform)
print('token ids = \n%s'%data_try[0][0])
print('segment ids = \n%s'%data_try[0][1])
print('valid length = \n%s'%data_try[0][2])

try_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_try],
                      batch_size=batch_size,
                      shuffle=False)
try_dataloader = mx.gluon.data.DataLoader(data_try, batch_sampler=try_sampler)



"""Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce's short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."""	0

"""Afraid of the Dark left me with the impression that several different screenplays were written, all too short for a feature length film, then spliced together clumsily into this Frankenstein's monster.<br /><br />At hi

In [18]:
ctx=mx.gpu(0)
for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(try_dataloader):

        # Load the data to the GPU
        token_ids = token_ids.as_in_context(ctx)
        valid_length = valid_length.as_in_context(ctx)
        segment_ids = segment_ids.as_in_context(ctx)

        # Forward computation
        out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
        
        print(out)
print("sentence","polarity")
output=out.argmax(axis=1)
for item in range(len(output)):
  if(output[item]==1):
    print(" ",item,"     positive")
  else:
    print(" ",item,"     negative")


[[-2.2869647  2.6782973]
 [ 2.6425257 -2.273133 ]]
<NDArray 2x2 @gpu(0)>
sentence polarity
  0      positive
  1      negative
